## Backtesting trading strategies

In [2]:
import pandas as pd 
import yfinance as yf
import sys
sys.path.append('../src')
from data import load_preprocessed_data, load_close_data, load_data
from strategies import MACD
from hydra import initialize, compose
import talib
import numpy as np
import multiprocessing
from golden_death_cross import GoldenDeathCross

sys.path.append("../")
import conf

%load_ext autoreload
%autoreload 2

In [24]:
PREPROCESSED_DATAPATH = conf.backtest_conf["data"]["preprocessed_datapath"]
# df = load_data(PREPROCESSED_DATAPATH,['_Close','_Open'])
df = load_data(PREPROCESSED_DATAPATH,['_Close'])
df.head()

,Date,AIR_Close,ABM_Close,AEG_Close,AMN_Close,AOS_Close,ASA_Close,T_Close,AZZ_Close,ANF_Close,...,WY_Close,WTM_Close,WMB_Close,WOLF_Close,WKC_Close,WWE_Close,WOR_Close,XRX_Close,YUM_Close,ZBH_Close
0,2002-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-01-02,9.20,15.380,25.903847,27.400000,3.191667,6.633333,30.135952,5.2625,26.809999,...,53.660000,350.000000,21.170744,29.700001,4.5600,13.50,14.00,27.457182,8.625090,29.349516
2,2002-01-03,9.66,15.320,26.000000,27.250000,3.268333,6.600000,30.339878,5.2625,28.080000,...,53.990002,346.899994,20.869120,32.330002,4.5375,13.71,14.10,27.404480,8.806614,29.165049
3,2002-01-04,9.84,15.605,25.038462,26.969999,3.380000,6.580000,30.203926,5.3000,29.820000,...,55.520000,340.000000,20.314783,32.450001,4.5750,13.74,14.45,26.482212,9.148095,28.689320
4,2002-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df.iloc[5:20]

,Date,AIR_Close,ABM_Close,AEG_Close,AMN_Close,AOS_Close,ASA_Close,T_Close,AZZ_Close,ANF_Close,...,WY_Close,WTM_Close,WMB_Close,WOLF_Close,WKC_Close,WWE_Close,WOR_Close,XRX_Close,YUM_Close,ZBH_Close
5,2002-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2002-01-07,9.70,15.300,24.576923,26.850000,3.361667,6.750000,30.143505,5.3000,29.049999,...,55.349998,340.0,20.379999,32.000000,4.5500,13.70,14.57,26.034256,9.318835,28.689320
7,2002-01-08,9.85,15.020,24.269232,26.500000,3.216667,6.880000,30.015106,5.2775,29.080000,...,54.290001,340.0,20.763144,31.639999,4.6425,13.70,14.08,26.218710,9.498562,29.611650
8,2002-01-09,9.85,15.080,23.942308,25.690001,3.225000,6.873333,28.829306,5.2525,27.740000,...,54.150002,338.5,20.869120,31.219999,4.8725,13.60,14.05,25.718050,9.507549,29.805824
9,2002-01-10,9.70,15.240,24.009615,25.209999,3.250000,7.003333,29.078550,5.2500,27.090000,...,53.369999,337.0,20.754992,30.650000,4.8200,13.68,14.22,25.691700,9.588426,29.669903
10,2002-01-11,9.55,14.950,24.096153,25.180000,3.283333,6.983333,28.572508,5.3125,25.959999,...,53.389999,334.5,20.787600,23.600000,4.8700,13.85,14.20,25.480896,9.545291,29.058252
11,2002-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2002-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2002-01-14,9.09,14.885,23.115385,24.940001,3.325000,7.033333,28.708460,5.2000,25.459999,...,52.110001,340.0,20.494127,21.309999,4.7750,13.35,13.84,25.401846,9.478792,30.883495
14,2002-01-15,8.92,14.875,23.307692,24.049999,3.300000,7.210000,28.542295,5.2725,24.969999,...,51.599998,335.0,20.151745,21.030001,4.5375,13.12,13.86,26.218710,9.579439,30.339806


In [ ]:
RAW_DATAPATH = conf.backtest_conf["data"]["raw_datapath"]
df = load_close_data(RAW_DATAPATH)
df.head()

,Date,AHC_Open,AHC_High,AHC_Low,AHC_Close,AHC_Adj Close,AHC_Volume,AIR_Open,AIR_High,AIR_Low,...,ZUO_Low,ZUO_Close,ZUO_Adj Close,ZUO_Volume,ZYME_Open,ZYME_High,ZYME_Low,ZYME_Close,ZYME_Adj Close,ZYME_Volume
0,2002-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-01-02,NaN,NaN,NaN,NaN,NaN,NaN,9.00,9.23,8.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-01-03,NaN,NaN,NaN,NaN,NaN,NaN,9.30,9.68,9.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-01-04,NaN,NaN,NaN,NaN,NaN,NaN,9.66,9.90,9.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2002-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
yf.download('A')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-11-18,32.546494,35.765381,28.612303,31.473534,26.794762,62546380
1999-11-19,30.713518,30.758226,28.478184,28.880545,24.587238,15234146
1999-11-22,29.551144,31.473534,28.657009,31.473534,26.794762,6577870
1999-11-23,30.400572,31.205294,28.612303,28.612303,24.358875,5975611
1999-11-24,28.701717,29.998213,28.612303,29.372318,25.005909,4843231
...,...,...,...,...,...,...
2023-07-03,120.019997,120.849998,119.220001,119.320000,119.320000,996200
2023-07-05,118.949997,120.279999,118.610001,119.500000,119.500000,1246200
2023-07-06,116.360001,118.889999,115.660004,118.430000,118.430000,1626600


In [2]:
# load close data
with initialize(version_base=None, config_path="../conf"):
    datapipeline_cfg = compose(config_name='datapipeline')
stock_data = load_close_data(datapipeline_cfg)
stock_data.head()

,Date,A_Close,AA_Close,AAIC_Close,AAP_Close,AAT_Close,AB_Close,ABB_Close,ABC_Close,ABEV_Close,...,X_Close,XOM_Close,XOXO_Close,XPO_Close,XRM_Close,XRX_Close,XYL_Close,YUM_Close,ZBH_Close,ZTR_Close
0,2002-01-02,20.922747,85.739037,108.000000,15.926667,21.250000,48.349998,9.89,15.7125,0.6,...,17.500000,39.599998,0.47,1.729270,240.000000,27.457182,24.25,8.625090,29.349516,28.400000
1,2002-01-03,22.246065,86.844421,116.000000,15.100000,21.309999,48.889999,10.61,15.4250,0.6,...,18.129999,39.660000,0.46,1.729270,233.000000,27.404480,25.35,8.806614,29.165049,28.559999
2,2002-01-04,23.447783,89.631897,119.599998,14.166667,21.370001,50.049999,11.11,15.0375,0.6,...,18.450001,40.000000,0.59,1.729270,233.000000,26.482212,25.16,9.148095,28.689320,28.799999
3,2002-01-07,23.354794,91.698479,117.000000,13.933333,21.250000,50.299999,10.70,14.5125,0.6,...,18.400000,39.650002,0.54,1.798440,233.199997,26.034256,25.24,9.318835,28.689320,28.840000
4,2002-01-08,23.426323,89.728020,123.000000,13.700000,21.209999,50.130001,10.83,14.6050,0.6,...,18.410000,39.700001,0.59,1.839943,235.000000,26.218710,24.75,9.498562,29.611650,28.840000


In [7]:
# load preprocessed data
processed_data = load_preprocessed_data(datapipeline_cfg)
processed_data.head()

c:\Users\User\Desktop\side projects\backtesting\notebooks\../src\data.py:23: DtypeWarning: Columns (5,10,12,14,15,17,23,27,29,33,34,36,38,39,41,47,51,63,64,67,76,77,78,79,80,83,85,86,95,99,103,110,111,113,117,124,126,131,132,134,139,141,143,147,153,154,155,156,157,164,167,169,175,176,177,179,181,186,188,191,197,199,200,203,204,207,208,209,211,214,217,220,221,225,229,238,241,243,245,247,248,258,259,261,264,268,273,275,276,282,284,287,289,290,295,297,301,303,304,306,309,318,322,323,335,336,337,341,345,349,350,351,352,357,358,362,363,364,365,367,369,371,372,373,374,380,392,395,396,398,399,400,401,403,404,405,406,407,411,412,415,420,424,425,428,429,430,431,433,435,436,437,450,451,453,454,456,458,459,460,463,464,466,468,469,472,473,474,476,477,479,480,481,486,492,493,494,495,496,497,501,505,509,511,512,513,514,516,517,518,519,521,522,525,526,530,531,534,538,540,545,546,547,548,555,558,562,572,574,575,576,577,578,584,586,589,591,592,593,598,601,602,603,604,609,611,613,615,617,621,628,632,638

Date                                                              \
            A          AA        AAIC         AAP         AAT          AB   
0  2002-01-02  2002-01-02  2002-01-02  2002-01-02  2011-01-13  2002-01-02   
1  2002-01-03  2002-01-03  2002-01-03  2002-01-03  2011-01-14  2002-01-03   
2  2002-01-04  2002-01-04  2002-01-04  2002-01-04  2011-01-18  2002-01-04   
3  2002-01-07  2002-01-07  2002-01-07  2002-01-07  2011-01-19  2002-01-07   
4  2002-01-08  2002-01-08  2002-01-08  2002-01-08  2011-01-20  2002-01-08   

                                                   ...    Volume              \
          ABB         ABC        ABEV         ABG  ...         X         XOM   
0  2002-01-02  2002-01-02  2002-01-02  2002-03-21  ...  835200.0  11261900.0   
1  2002-01-03  2002-01-03  2002-01-03  2002-03-22  ...  702500.0  10128200.0   
2  2002-01-04  2002-01-04  2002-01-04  2002-03-25  ...  731700.0   8052100.0   
3  2002-01-07  2002-01-07  2002-01-07  2002-03-26  ...  403700.0   9945700.0   
4  2002-01-08  2002-01-08  2002-01-08  2002-03-27  ...  473500.0   8278000.0   

                                                                          \
      XOXO      XPO       XRM        XRX       XYL        YUM        ZBH   
0   1000.0  36793.0  196950.0  3233074.0  396800.0  6378570.0   818850.0   
1   2300.0   7228.0   77500.0  1874275.0  402600.0  5461066.0  1058016.0   
2  33500.0  20963.0    7515.0  3601721.0  583400.0  8121214.0   810198.0   
3    100.0   5494.0    5250.0  6261295.0  423500.0  8309834.0   695044.0   
4    500.0  20167.0    9465.0  2192865.0  722200.0  8023288.0   898160.0   

            
       ZTR  
0  22225.0  
1  17825.0  
2  18925.0  
3  31700.0  
4  20450.0  

[5 rows x 9975 columns]

In [3]:
df = stock_data.copy()

### Size of data 

In [12]:
num_companies = len(df.filter(regex='Close').columns.tolist())
print(f'20 years of daily data from {num_companies} companies')

20 years of daily data from 1429 companies


### Missing values

In [13]:
# Top 10 tickers with missing values 
missing_values = df.isnull().any().sum()
print(f'Number of tickers with missing values: {missing_values}\
      \nTop 10 tickers with missing values:')
df.isnull().sum().sort_values(ascending=False)[:10]

Number of tickers with missing values: 477      
Top 10 tickers with missing values:


CIVI_Close    2508
VTLE_Close    2508
CPRI_Close    2508
BUI_Close     2493
APTV_Close    2489
VAC_Close     2482
TTP_Close     2474
GNE_Close     2473
UI_Close      2465
XYL_Close     2464
dtype: int64

### MACD

In [25]:
macd_positions = MACD(df) 
positions = macd_positions.get_entry_exit_signal()
positions

,AAIC_Entry,AAIC_Exit,AAP_Entry,AAP_Exit,AA_Entry,AA_Exit,ABB_Entry,ABB_Exit,ABC_Entry,ABC_Exit,...,XRX_Entry,XRX_Exit,X_Entry,X_Exit,YUM_Entry,YUM_Exit,ZBH_Entry,ZBH_Exit,ZTR_Entry,ZTR_Exit
Date,,,,,,,,,,,,,,,,,,,,,
2002-01-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-02,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-03,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-04,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2002-01-05,nan,exit_long,nan,exit_short,nan,exit_long,nan,exit_long,nan,exit_short,...,nan,exit_short,nan,exit_long,nan,exit_long,nan,exit_short,nan,exit_long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2022-12-28,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2022-12-29,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [37]:
positions['X_Entry'].unique()

array(['nan'], dtype=object)

In [6]:
macd_positions = pd.read_csv('../data/intermediate/macd_positions.csv')
macd_positions.drop(['Unnamed: 0'], axis=1,inplace=True)
macd_positions.head()

C:\Users\User\AppData\Local\Temp\ipykernel_2436\1543702493.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

,A_Position,A_Entry,A_Exit,AA_Position,AA_Entry,AA_Exit,AAIC_Position,AAIC_Entry,AAIC_Exit,AAP_Position,...,XYL_Exit,YUM_Position,YUM_Entry,YUM_Exit,ZBH_Position,ZBH_Entry,ZBH_Exit,ZTR_Position,ZTR_Entry,ZTR_Exit
0,long,2002-03-11,2002-03-19,long,2002-03-19,2002-03-20,long,2002-03-08,2002-03-11,long,...,2002-04-01,long,2002-02-26,2002-02-27,long,2002-04-29,2002-05-01,long,2002-03-06,2002-03-07
1,long,2002-03-21,2002-03-22,long,2002-06-20,2002-06-21,long,2002-04-12,2002-04-15,long,...,2002-05-09,long,2002-04-10,2002-04-11,long,2002-06-20,2002-06-28,long,2002-03-21,2002-03-26
2,long,2002-04-29,2002-05-01,long,2002-07-29,2002-07-30,long,2002-04-30,2002-05-01,long,...,2002-08-02,long,2002-04-23,2002-04-24,long,2002-07-26,2002-07-29,long,2002-03-26,2002-03-28
3,long,2002-06-13,2002-06-14,long,2002-09-12,2002-09-17,long,2002-05-20,2002-06-04,long,...,2002-08-21,long,2002-05-16,2002-05-20,long,2002-09-09,2002-09-19,long,2002-04-15,2002-04-17
4,long,2002-08-01,2002-08-07,long,2002-09-30,2002-10-03,long,2002-06-11,2002-06-12,long,...,2002-08-21,long,2002-06-11,2002-06-19,long,2002-09-23,2002-09-24,long,2002-07-08,2002-07-18


### Golden death cross

In [7]:
golden_death_cross = GoldenDeathCross(df)
output = golden_death_cross.get_cross(df['A_Close'])
output[output['cross']=='golden']

,Date,cross,long_term_ma
351,2003-05-27,golden,10.832940
880,2005-06-30,golden,16.508977
1243,2006-12-07,golden,24.061051
1632,2008-06-26,golden,24.671781
1890,2009-07-07,golden,13.468348
2223,2010-10-29,golden,22.533691
2556,2012-02-27,golden,28.641774
2778,2013-01-15,golden,28.082582
3268,2014-12-24,golden,40.388131
3333,2015-03-31,golden,40.602784


### Optimising MACD strategy

In [15]:
# try and optimise 

def get_entry_exit_position(enter: pd.Series, exit: pd.Series):
    position_df = pd.DataFrame(columns=["Position", "Entry", "Exit"])

    # Process long positions
    entry_long_idx = pd.Series(enter[enter == "enter_long"].index)
    exit_long_idx = pd.Series(exit[exit == "exit_long"].index)

    long_positions = pd.concat([entry_long_idx, exit_long_idx])
    long_positions = long_positions.sort_values()

    # Get exit positions for long positions
    exit_long_positions = long_positions[long_positions.isin(exit_long_idx)]
    # exit_long_positions = exit_long_positions[1:]  # Exclude the first entry position

    return exit_long_positions

get_entry_exit_position(positions[column_pairs[0][0]],positions[column_pairs[0][1]])

0       2002-01-07
1       2002-01-08
2       2002-01-09
3       2002-01-18
4       2002-01-23
           ...    
1975    2022-11-28
1976    2022-12-02
1977    2022-12-05
1978    2022-12-20
1979    2022-12-30
Name: Date, Length: 2154, dtype: object

In [6]:
# timeit
import timeit
mysetup = """
import pandas as pd
import sys
sys.path.append('../src')
from data import load_preprocessed_data, load_close_data
from strategies import MACD
from hydra import initialize, compose
import numpy as np
with initialize(version_base=None, config_path="../conf"):
    datapipeline_cfg = compose(config_name='datapipeline')
stock_data = load_close_data(datapipeline_cfg)
df = stock_data.copy()
macd_positions = MACD(df) 
positions = macd_positions.apply()
"""
mycode = '''
columns = positions.columns
for i in range(0, len(columns), 2):
    col1 = columns[i]
    col2 = columns[i + 1]
    col_name = f'{col1.split("_")[0]}_Exit_Position'
    exit_position = macd_positions.get_exit_position(positions[col1],positions[col2]).rename(col_name)
    positions = pd.concat([positions, exit_position], axis=1)
'''
print(timeit.timeit(setup=mysetup,
                    stmt=mycode,
                    number=1))

1050.2912589


### Get macd entry and exit positions and save as csv file
Dask parallel processing saved about 4min 

In [29]:
import dask
def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
lazy_list = []
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
for pair in column_pairs:
    exit_position = dask.delayed(lazy_function)(pair)
    lazy_list.append(exit_position)
exit_position = dask.compute(*lazy_list)

In [31]:
# merge into 1 dataframe and save it
tickers = [col.split('_')[0] for col in df.iloc[:, 1:].columns]
for i in range(len(exit_position)):
    exit_position[i].rename(columns=lambda x: tickers[i] + '_' + x, inplace=True)
merged_df = pd.concat(exit_position, axis=1)

['A',
 'AA',
 'AAIC',
 'AAP',
 'AAT',
 'AB',
 'ABB',
 'ABC',
 'ABEV',
 'ABG',
 'ABM',
 'ABR',
 'ABT',
 'ACCO',
 'ACM',
 'ACN',
 'ACR',
 'ADC',
 'ADM',
 'ADX',
 'AEE',
 'AEG',
 'AEL',
 'AEM',
 'AEO',
 'AEP',
 'AER',
 'AES',
 'AET',
 'AFB',
 'AFG',
 'AFL',
 'AFT',
 'AG',
 'AGCO',
 'AGI',
 'AGM',
 'AGO',
 'AGRO',
 'AGX',
 'AHT',
 'AIG',
 'AIN',
 'AIR',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AJRD',
 'AKR',
 'AL',
 'ALB',
 'ALE',
 'ALG',
 'ALK',
 'ALL',
 'ALR',
 'ALV',
 'ALX',
 'AME',
 'AMG',
 'AMN',
 'AMP',
 'AMRC',
 'AMT',
 'AN',
 'ANDV',
 'ANF',
 'AON',
 'AORT',
 'AOS',
 'AP',
 'APA',
 'APD',
 'APH',
 'APO',
 'APTV',
 'AQN',
 'ARC',
 'ARCO',
 'ARE',
 'ARGO',
 'ARI',
 'ARL',
 'AROC',
 'ARR',
 'ARW',
 'ASA',
 'ASB',
 'ASG',
 'ASGN',
 'ASH',
 'ASR',
 'ASXC',
 'ATCO',
 'ATGE',
 'ATI',
 'ATO',
 'ATOR',
 'ATR',
 'AUB',
 'AUD',
 'AUY',
 'AVA',
 'AVB',
 'AVD',
 'AVNT',
 'AVY',
 'AWF',
 'AWI',
 'AWK',
 'AWR',
 'AX',
 'AXL',
 'AXP',
 'AXR',
 'AXS',
 'AYI',
 'AZO',
 'AZZ',
 'B',
 'BA',
 'BAC',
 'BAH',
 

In [51]:
from dask.dataframe import from_pandas
# ddf = from_pandas(positions, npartitions=1425)

def lazy_function(partition):
    col1,col2 = partition.columns[0],partition.columns[1]
    exit_position = macd_positions.get_exit_position(partition[col1],partition[col2])
    return exit_position

# Apply split_dataframe_by_columns to each partition
partitions = [positions.iloc[:, i:i+2] for i in range(0, positions.shape[1], 2)]
ddf_partitions = [from_pandas(partition, npartitions=1) for partition in partitions]
result = [partition.map_partitions(lazy_function).compute() for partition in ddf_partitions]
# result = new_df.compute()
print(result)


KeyboardInterrupt: 

In [50]:
import multiprocessing as mp
pool = mp.Pool()

def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
lazy_list = []
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
for pair in column_pairs:
    result = pool.apply_async(lazy_function, args=(pair,))
    lazy_list.append(result)
pool.close()
pool.join()

[     AAIC_Entry   AAIC_Exit
 0           nan         nan
 1           nan         nan
 2           nan         nan
 3           nan   exit_long
 4           nan   exit_long
 ...         ...         ...
 5282        nan  exit_short
 5283        nan         nan
 5284        nan  exit_short
 5285        nan  exit_short
 5286        nan   exit_long
 
 [5287 rows x 2 columns],
      AAP_Entry    AAP_Exit
 0          nan         nan
 1          nan         nan
 2          nan         nan
 3          nan  exit_short
 4          nan  exit_short
 ...        ...         ...
 5282       nan   exit_long
 5283       nan   exit_long
 5284       nan   exit_long
 5285       nan   exit_long
 5286       nan   exit_long
 
 [5287 rows x 2 columns],
      AAT_Entry    AAT_Exit
 0          nan         nan
 1          nan         nan
 2          nan         nan
 3          nan   exit_long
 4          nan  exit_short
 ...        ...         ...
 5282       nan         nan
 5283       nan         nan
 5284   

In [14]:
import concurrent.futures
def lazy_function(pair):
    col1,col2 = pair[0],pair[1]
    exit_position = macd_positions.get_entry_exit_position(positions[col1],positions[col2])
    return exit_position
column_pairs = [(positions.columns[i], positions.columns[i+1]) for i in range(0, len(positions.columns), 2)]
with concurrent.futures.ProcessPoolExecutor() as executor:
    # can replace ProcessPoolExecutor with ThreadPoolExecutor
    result = list(executor.map(lazy_function, [pair for pair in column_pairs]))

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [31]:
# convert to dask df
from dask.dataframe import from_pandas
ddf = from_pandas(positions, npartitions=2)
column_pairs = [(ddf.columns[i], ddf.columns[i+1]) for i in range(0, len(ddf.columns), 2)]
for pair in column_pairs:
pd.concat([positions, exit_position], axis=1)


In [30]:
macd_df = pd.DataFrame()
test_df = yf.download('A', start="2002-01-01",
                   end="2003-01-01", interval = "1d",
                   )
macd_df["rsi"] = talib.RSI(test_df['Close'], timeperiod=2)
macd_df.iloc[2:10]

[*********************100%***********************]  1 of 1 completed


,rsi
Date,
2002-01-04,100.000000
2002-01-07,93.139912
2002-01-08,93.794802
2002-01-09,37.679751
2002-01-10,27.234886
2002-01-11,10.859347
2002-01-14,5.179905
2002-01-15,9.747900


In [46]:
macd_df = pd.DataFrame()
test_df = yf.download('A', start="2002-01-01",
                   end="2003-01-01", interval = "1d",
                   )
macd = talib.MACD(test_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9) 
macd_df['histogram'] = macd[2] # macd line - signal line
macd_df['macd'] = macd[0]

#entry where 1 = enter and 0 = do not enter trade
macd_df['long_signal'] = np.where((macd_df['histogram']>0) &(macd_df['histogram'].shift(1)<=0),1,0)
macd_df['short_signal'] = np.where((macd_df['histogram']<0) &(macd_df['histogram'].shift(1)>=0),1,0)
# trading positions where you enter the trade 1 day after the signal (assuming that the closing price is used to calculate the EMA)
macd_df['long_position'] = np.where(macd_df['long_signal'].shift(1)==1,1,0) 
macd_df['short_position'] = np.where(macd_df['short_signal'].shift(1)==1,1,0)
macd_df['position'] = np.where(macd_df['long_position'] == 1, 'long', np.where(macd_df['short_position'] == 1, 'short', np.nan))
macd_df


[*********************100%***********************]  1 of 1 completed


,histogram,macd,long_signal,short_signal,long_position,short_position,position
Date,,,,,,,
2002-01-02,NaN,NaN,0,0,0,0,nan
2002-01-03,NaN,NaN,0,0,0,0,nan
2002-01-04,NaN,NaN,0,0,0,0,nan
2002-01-07,NaN,NaN,0,0,0,0,nan
2002-01-08,NaN,NaN,0,0,0,0,nan
...,...,...,...,...,...,...,...
2002-12-24,-0.034280,0.384308,0,0,0,0,nan
2002-12-26,-0.018510,0.395451,0,0,0,0,nan
2002-12-27,-0.015994,0.393969,0,0,0,0,nan


In [62]:
macd_df = pd.DataFrame()
test_df = yf.download('AMZN', start="2017-01-01",
                   end="2017-04-30", interval = "1d",
                   )
macd = talib.MACD(test_df['Close'], fastperiod=12, slowperiod=26, signalperiod=9) 
macd_df['histogram'] = macd[2] # macd line - signal line
macd_df['macd'] = macd[0]

#entry where 1 = enter and 0 = do not enter trade
macd_df['long_signal'] = np.where((macd_df['histogram']>0) &(macd_df['histogram'].shift(1)<=0),1,0)
macd_df['short_signal'] = np.where((macd_df['histogram']<0) &(macd_df['histogram'].shift(1)>=0),1,0)
# trading positions where you enter the trade 1 day after the signal (assuming that the closing price is used to calculate the EMA)
macd_df['long_position'] = np.where(macd_df['long_signal'].shift(1)==1,1,0) 
macd_df['short_position'] = np.where(macd_df['short_signal'].shift(1)==1,1,0)
macd_df['position'] = np.where(macd_df['long_position'] == 1, 'long', np.where(macd_df['short_position'] == 1, 'short', np.nan))
# drop unnecessary columns to free space 
macd_df = macd_df.drop(['long_position', 'short_position','long_signal','short_signal'], axis=1)
# macd_df['position'] = np.where((macd_df['long_position']==1)|(macd_df['short_position']==1),1,0)

#exit (2-day RSI of single day is greater than 65 for long positions, and less than 35 for short positions)
macd_df['Close'] = test_df['Close']
macd_df['Date'] = macd_df.index
macd_df.reset_index(inplace=True,drop=True)
macd_df['rsi'] = talib.RSI(macd_df['Close'],timeperiod=2)
macd_df['entry_date'] = macd_df.apply(lambda _: '', axis=1)
for i in macd_df.loc[macd_df['position']==1].index:
    macd_df['entry_date'].iloc[i] = macd_df['Date'].iloc[i]
macd_df['exit_long'] = np.where(macd_df['rsi'].shift(1)>65,1,0) #signal to exit is generated when 2-period RSI above 65, but exit trade is only executed on following day.
macd_df['exit_short'] = np.where(macd_df['rsi'].shift(1)<35,1,0)
macd_df['exit_date'] = ''
# exit long positions
for i in macd_df.loc[macd_df['long_position']==1].index:
    new_macd_df = macd_df.loc[i+2:]
    if len(new_macd_df) == 0: #index out of range, so use last date as exit
        macd_df['exit_date'].iloc[i] = macd_df['Date'].iloc[-1]
    for j in range(len(new_macd_df)):
        if new_macd_df['exit_long'].iloc[j]==1:
            macd_df['exit_date'].iloc[i] = new_macd_df['Date'].iloc[j]
            break
        else:
            macd_df['exit_date'].iloc[i] = macd_df['Date'].iloc[-1]
# exit short positions
for i in macd_df.loc[macd_df['short_position']==1].index:
    new_macd_df = macd_df.loc[i+2:]
    if len(new_macd_df) == 0: #index out of range, so use last date as exit
        macd_df['exit_date'].iloc[i] = macd_df['Date'].iloc[-1]
    for j in range(len(new_macd_df)):
        if new_macd_df['exit_short'].iloc[j]==1:
            macd_df['exit_date'].iloc[i] = new_macd_df['Date'].iloc[j]
            break
        else:
            macd_df['exit_date'].iloc[i] = macd_df['Date'].iloc[-1]

### Profit
macd_df['profit'] = ''
macd_df['winlose'] = ''

#long profit
for i in macd_df[macd_df['long_position']==1].index:
    start_price = float(macd_df[macd_df['Date']==macd_df['entry_date'].iloc[i]]['Close'])
    end_price = float(macd_df[macd_df['Date']==macd_df['exit_date'].iloc[i]]['Close'])    
    profit = 1+(end_price-start_price)/start_price
    if profit > 1:
        winlose = 1 
    else:
        winlose = 0
    macd_df['profit'].iloc[i] = profit
    macd_df['winlose'].iloc[i] = winlose

#short profit
for i in macd_df[macd_df['short_position']==1].index:
    start_price = float(macd_df[macd_df['Date']==macd_df['exit_date'].iloc[i]]['Close'])
    end_price = float(macd_df[macd_df['Date']==macd_df['entry_date'].iloc[i]]['Close'])  
    profit = 1+(end_price-start_price)/start_price
    if profit > 1:
        winlose = 1 
    else:
        winlose = 0
    macd_df['profit'].iloc[i] = profit
    macd_df['winlose'].iloc[i] = winlose

return macd_df[macd_df['position']==1]


[*********************100%***********************]  1 of 1 completed


KeyError: 'long_position'

In [35]:
index = macd_df[macd_df['Date'] == '2017-03-30'].index[0]
macd_df.loc[index:]

,histogram,macd,long_signal,short_signal,long_position,short_position,position,Close,Date,rsi,entry_date,exit_long,exit_short,exit_date
60,0.065857,0.329368,0,0,1,0,1,43.817001,2017-03-30,98.265638,2017-03-30 00:00:00,1,0,2017-04-03 00:00:00
61,0.126896,0.422131,0,0,0,0,0,44.327000,2017-03-31,99.295241,,1,0,
62,0.171669,0.509822,0,0,0,0,0,44.575500,2017-04-03,99.553532,,1,0,
63,0.236534,0.633820,0,0,0,0,0,45.341499,2017-04-04,99.863023,,1,0,
64,0.268987,0.733520,0,0,0,0,0,45.464001,2017-04-05,99.887881,,1,0,
65,0.235892,0.759398,0,0,0,0,0,44.914001,2017-04-06,37.985993,,1,0,
66,0.187161,0.757457,0,0,0,0,0,44.743999,2017-04-07,27.464372,,0,0,
67,0.180408,0.795806,0,0,0,0,0,45.352001,2017-04-10,75.669417,,0,1,
68,0.146174,0.798116,0,0,0,0,0,45.118000,2017-04-11,50.060957,,1,0,
69,0.091551,0.766380,0,0,0,0,0,44.811501,2017-04-12,26.535672,,0,0,


In [25]:
macd_df[macd_df['long_position']==1]

,histogram,macd,long_signal,short_signal,long_position,short_position,position,Close,Date,rsi,entry_date,exit_long,exit_short,exit_date
60,0.065857,0.329368,0,0,1,0,1,43.817001,2017-03-30,98.265638,2017-03-30 00:00:00,1,0,2017-04-03 00:00:00


In [26]:
macd_df.loc['2017-03-27':]

,histogram,macd,long_signal,short_signal,long_position,short_position,position
Date,,,,,,,
2017-03-27,-0.105279,0.156655,0,0,0,0,0
2017-03-28,-0.070606,0.173677,0,0,0,0,0
2017-03-29,0.011058,0.258105,1,0,0,0,0
2017-03-30,0.065857,0.329368,0,0,1,0,1
2017-03-31,0.126896,0.422131,0,0,0,0,0
2017-04-03,0.171669,0.509822,0,0,0,0,0
2017-04-04,0.236534,0.633820,0,0,0,0,0
2017-04-05,0.268987,0.733520,0,0,0,0,0
2017-04-06,0.235892,0.759398,0,0,0,0,0


In [5]:
aapl = yf.Ticker("aapl")
aapl.dividends

Date
1987-05-11 00:00:00-04:00    0.000536
1987-08-10 00:00:00-04:00    0.000536
1987-11-17 00:00:00-05:00    0.000714
1988-02-12 00:00:00-05:00    0.000714
1988-05-16 00:00:00-04:00    0.000714
                               ...   
2022-02-04 00:00:00-05:00    0.220000
2022-05-06 00:00:00-04:00    0.230000
2022-08-05 00:00:00-04:00    0.230000
2022-11-04 00:00:00-04:00    0.230000
2023-02-10 00:00:00-05:00    0.230000
Name: Dividends, Length: 78, dtype: float64

In [7]:
data = pd.DataFrame()
for i in ['AMZN']:
    stock = yf.download(i, start="2017-01-01",
                   end="2017-04-30", interval = "1d",
                   )
    stock = stock.dropna().reset_index() 
    #stock = pd.MultiIndex.from_frame(stock,names=[])
stock

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,37.896000,37.938000,37.384998,37.683498,37.683498,70422000
1,2017-01-04,37.919498,37.984001,37.709999,37.859001,37.859001,50210000
2,2017-01-05,38.077499,39.119999,38.013000,39.022499,39.022499,116602000
3,2017-01-06,39.118000,39.972000,38.924000,39.799500,39.799500,119724000
4,2017-01-09,39.900002,40.088501,39.588501,39.846001,39.846001,68922000
...,...,...,...,...,...,...,...
76,2017-04-24,45.433998,45.499500,45.191002,45.370499,45.370499,62458000
77,2017-04-25,45.352001,45.473999,45.150002,45.381001,45.381001,67612000
78,2017-04-26,45.514999,45.787498,45.377998,45.464500,45.464500,52178000
79,2017-04-27,45.719501,46.092999,45.605499,45.918999,45.918999,106110000
